In [1]:
import json
import geopandas as gpd
import numpy as np
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [3]:
repro_crs = '+proj=cea'

In [4]:
build = gpd.GeoDataFrame.from_postgis('SELECT * from afg_buildings_microsoft', con)

#build = gpd.read_file(r'D:\iMMAP\proj\ASDC\data\Flood\build-sample-large-v2.shp')

In [5]:
# Reproject to a CRS
build = build.to_crs(repro_crs)

In [6]:
### Convert building polygons to centroid points

# Convert building geodataframe to geoseries
build_centroids = build.geometry

# Create a geoseries with just the centroids
build_centroids = build_centroids.centroid

# Convert geoseries to geodataframe
build_centroids = gpd.GeoDataFrame(build_centroids)

In [7]:
# Rename and set geometry column
build_centroids.rename(columns = {'geometry': 'geom'}, inplace = True)
build_centroids.rename(columns = {0: 'geom'}, inplace = True)
build_centroids = build_centroids.set_geometry("geom")

In [8]:
# Reproject back to 4326
build_centroids = build_centroids.to_crs('epsg:4326')

In [9]:
build_centroids.to_postgis("afg_buildings_microsoft_centroids", con)